In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')

%matplotlib inline

In [59]:
# read data
data_treino = pd.read_csv(
    'data/credit_train_label.csv'
)

data_resposta = pd.read_csv(
    'data/credit_test_features.csv'
)

In [60]:
data_treino.drop('X', axis=1, inplace=True)
data_resposta.drop('X', axis=1, inplace=True)

In [61]:
X = data_treino.iloc[:, 1:]
y = data_treino.iloc[:, 0]

In [63]:
# Windsor
windsor = {}
for feature in X.columns:
    windsor[feature] = X[feature].quantile(0.99)
    X.loc[X[feature] > windsor[feature], feature] = windsor[feature]
    data_resposta.loc[data_resposta[feature] > windsor[feature], feature] = windsor[feature]


In [64]:
monthly_income_median = X['MonthlyIncome'].median()
number_of_dependents = X['NumberOfDependents'].median()

In [65]:
X['MonthlyIncome'].fillna(monthly_income_median, inplace=True)
X['NumberOfDependents'].fillna(number_of_dependents, inplace=True)

data_resposta['MonthlyIncome'].fillna(monthly_income_median, inplace=True)
data_resposta['NumberOfDependents'].fillna(number_of_dependents, inplace=True)

In [66]:
# from imblearn.over_sampling import SMOTE, RandomOverSampler

# sm = SMOTE(random_state=433)
# sm = RandomOverSampler(random_state=433)
# X_train, y_train = sm.fit_sample(X_train, y_train)

In [51]:
#modelo1
from sklearn.ensemble import GradientBoostingClassifier

params = {'n_estimators': 500, 'max_depth': 2, 'min_samples_split': 2,
          'learning_rate': 0.01, 'random_state': 42}

clf = GradientBoostingClassifier(**params)

clf.fit(X, y)
y_hat = clf.predict(data_resposta)
y_probs = clf.predict_proba(data_resposta)

In [67]:
#modelo 2
params = {
    'subsample': 1,
    'n_estimators': 300,
    'min_samples_split': 2,
    'max_depth': 3,
    'learning_rate': 0.1,
    'random_state': 42
}

clf = GradientBoostingClassifier(**params)

clf.fit(X, y)
y_hat = clf.predict(data_resposta)
y_probs = clf.predict_proba(data_resposta)

In [53]:
y_probs

array([[0.98623598, 0.01376402],
       [0.99184227, 0.00815773],
       [0.99563994, 0.00436006],
       ...,
       [0.98355338, 0.01644662],
       [0.98710024, 0.01289976],
       [0.97370682, 0.02629318]])

In [56]:
data_resposta['Predicted'] = y_probs[:,1]

In [57]:
data_resposta.to_csv('submission_pedro_correia2.csv', index=False) #0.86286